In [11]:
from openai import OpenAI

with open('API_KEY', 'r') as file:
    API_KEY = file.read().strip()

client = OpenAI(
    api_key=API_KEY
)
# defaults to getting the key using os.environ.get("OPENAI_API_KEY")


In [39]:
import pandas as pd

df = pd.read_csv('similar_sensitive_words.csv')

# each request should be just 10 terms
n = 10
df_chunks = [df[i:i + n] for i in range(0, df.shape[0], n)]

# to be carefull of token limits drop most rows
df_chunks = df_chunks[0:2]

# convert the list of chunks to a list of lists turned into strings with the similar_word column
requests = [[str(l) for l in df_chunk["similar_word"]] for df_chunk in df_chunks]
print(f"{requests[0]}")


['discriminatory', 'extremism', 'profiling', 'stereotyping', 'islamism', 'eugenics', 'orientations', 'colonialism', 'slur', 'predominance']


In [51]:
gpt3 = "gpt-3.5-turbo"
gpt4 = "gpt-4-turbo-preview"

completion = client.chat.completions.create(
  model=gpt4,
  # response_format={ "type": "json_object" }, # argument that forces the response to be a json object but this seems to not work on a list of words as requests
  messages=[
    {"role": "system", "content": """Your role is to assess the sensitivity of list of words provided in either German or English, assigning a sensitivity rating from 0 to 1, with 1 being very sensitive. Upon receiving a word, you will in one paragraph for each word propose how this word can be defined, pinpoint what is sensitive about the word, and discuss if possible 2 or 3 options for translating it between English and German. Each paragraph should be very short, spanning a maximum of 250 tokens in total. This requires concise yet insightful analysis, ensuring language is used respectfully and accurately, with a focus on cultural and contextual nuances.

Incorporate the principles and goals of macht.sprache. to support users working in German and English to translate more sensitively. This includes recognizing and addressing linguistic discrimination, finding expressions that de-normalize such discrimination, and fostering awareness for the sensitive handling of political terms in translations. Emphasize continuity, collaboration, creativity, and accessibility in this process. It's essential to understand that macht.sprache. cannot take responsibility for the sensitivity of human translators with political terms, emphasizing the importance of self-education.

Consider the perspectives provided by macht.sprache. on recognizing power and privileges, increasing awareness for justice, and choosing words that minimize harm over those that cause it. All translation decisions are political, and this perspective should guide the sensitivity assessment and translation options provided. The collaborative and ongoing nature of macht.sprache., its foundation in diverse expert contributions, and its commitment to pragmatism, accessibility, and creativity in translations are integral to your analysis and recommendations.

Outputs should be in JSON format with the following keys for each word is the requested list: "word", "sensitivity_rating", "definition", and "translations_options" with a list of 4 "options" and their "nuance"."""},
    {"role": "user", "content": f"{requests[1]}"},
  ]
)

print(completion.choices[0].message.content)

```json
[
    {
        "word": "totalitarianism",
        "sensitivity_rating": 0.7,
        "definition": "A system of government that is centralized and dictatorial and requires complete subservience to the state.",
        "translations_options": [
            {"options": "Totalitarismus", "nuance": "Direct translation capturing the concept universally."},
            {"options": "Diktatur", "nuance": "Emphasizes the dictatorial aspect, slightly less precise."},
            {"options": "autoritäres Regime", "nuance": "Highlights authoritarian nature, broader than 'Totalitarismus'."},
            {"options": "Staatsabsolutismus", "nuance": "Focuses on the absolute power of the state, historical nuance."}
        ]
    },
    {
        "word": "xenophobic",
        "sensitivity_rating": 0.9,
        "definition": "Having or showing a dislike of or prejudice against people from other countries.",
        "translations_options": [
            {"options": "fremdenfeindlich", "nuance": "

In [ ]:
    {"role": "system", "content": """Your role is to assess the sensitivity of words provided in either German or English, assigning a sensitivity rating from 0 to 1, with 1 being very sensitive. Upon receiving a word, you will in one paragraph for each word propose how this word can be defined, pinpoint what is sensitive about the word, and discuss if possible 2 or 3 options for translating it between English and German. Each paragraph should be very short, spanning a maximum of 150 tokens in total. This requires concise yet insightful analysis, ensuring language is used respectfully and accurately, with a focus on cultural and contextual nuances.

Incorporate the principles and goals of macht.sprache. to support users working in German and English to translate more sensitively. This includes recognizing and addressing linguistic discrimination, finding expressions that de-normalize such discrimination, and fostering awareness for the sensitive handling of political terms in translations. Emphasize continuity, collaboration, creativity, and accessibility in this process. It's essential to understand that macht.sprache. cannot take responsibility for the sensitivity of human translators with political terms, emphasizing the importance of self-education.

Consider the perspectives provided by macht.sprache. on recognizing power and privileges, increasing awareness for justice, and choosing words that minimize harm over those that cause it. All translation decisions are political, and this perspective should guide the sensitivity assessment and translation options provided. The collaborative and ongoing nature of macht.sprache., its foundation in diverse expert contributions, and its commitment to pragmatism, accessibility, and creativity in translations are integral to your analysis and recommendations.

Outputs should be in JSON format with four entries for each word: the word itself, its sensitivity rating, a definition, and 2 to 4 translation options with their relation."""},
    {"role": "user", "content": requests[0]},
  ]
)

In [50]:
# Specify the file name
file_name = "output.json"

# Use open() to open the file in write mode ('w')
# If the file does not exist, it will be created.
with open(file_name, 'w') as file:
    # Use the write() method to write the string to the file
    file.write(completion.choices[0].message.content)

# Open the file in append mode to add additional content instead of overwriting the content
# with open(file_name, 'a') as file:
#     file.write("\nThis is additional content.")

str